# FLSE distillation con GloVe o vocab filtrado

Notebook breve para probar:
- Teacher GloVe 6B (300d, inglés)
- Teacher fastText es filtrado (solo tokens alfabéticos) en un vocab recortado
- Entrenamiento 5 capas con temperaturas por capa
- Vecinos filtrados

## 0) Setup repo
Clona y prepara entorno editable.

In [ ]:
!git clone https://github.com/santiagoNieva/FLSE.git
%cd FLSE
!pip install -e .

## 1) Teacher GloVe 6B (300d)
Descarga con torchtext y guarda .npy (usa inglés).

In [ ]:
!pip install torchtext
import numpy as np
from torchtext.vocab import GloVe
glove = GloVe(name='6B', dim=300)
np.save('/content/FLSE/teacher_glove6B_300.npy', glove.vectors.numpy())
# vocab size ~400k, dim=300

## 2) Teacher fastText es filtrado (solo alfabético, recorte 100k)
Útil para reducir ruido de signos/siglas.

In [ ]:
!wget -O /content/cc.es.300.bin.gz https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz
!gunzip /content/cc.es.300.bin.gz

import fasttext, numpy as np
ft = fasttext.load_model('/content/cc.es.300.bin')
raw_words = ft.words
keep = [w for w in raw_words if w.isalpha() and len(w)>2][:100000]
vecs = np.vstack([ft.get_word_vector(w) for w in keep])
np.save('/content/FLSE/teacher_fasttext_es_100k_clean.npy', vecs)
np.save('/content/FLSE/teacher_fasttext_es_100k_vocab.npy', np.array(keep))
# vocab size 100k, dim=300

## 3) Entrenamiento (ejemplo con GloVe, 5 capas)
Ajusta temps/targets según necesidad.

In [ ]:
!python experiments/distill_playground.py \
  --teacher-path /content/FLSE/teacher_glove6B_300.npy \
  --vocab-size 400000 --teacher-dim 300 \
  --num-layers 5 --verts-per-layer 12 --dim 12 \
  --epochs 10 --batch-size 64 \
  --lambda-ent 8.0 \
  --lambda-entropies 1.0 1.5 2.5 3.0 3.0 \
  --target-entropies 1.6 1.0 0.8 0.5 0.35 \
  --logit-temps 1.0 1.5 2.0 2.5 2.7 \
  --device cuda \
  --save-path /content/FLSE/data/flse_glove5c.pt

## 4) Entrenamiento (ejemplo con fastText limpio 100k)
Mismo set de hiperparámetros.

In [ ]:
!python experiments/distill_playground.py \
  --teacher-path /content/FLSE/teacher_fasttext_es_100k_clean.npy \
  --vocab-size 100000 --teacher-dim 300 \
  --num-layers 5 --verts-per-layer 12 --dim 12 \
  --epochs 12 --batch-size 64 \
  --lambda-ent 9.0 \
  --lambda-entropies 1.0 1.5 2.5 3.0 3.0 \
  --target-entropies 1.6 1.0 0.8 0.5 0.35 \
  --logit-temps 1.0 1.5 2.0 2.5 2.7 \
  --device cuda \
  --save-path /content/FLSE/data/flse_fasttext_clean5c.pt

## 5) Cargar, vecinos filtrados y entropías
Reutiliza el código de carga/vecinos del otro notebook ajustando rutas/paths de teacher y vocab (para fastText limpio carga `teacher_fasttext_es_100k_vocab.npy`).